# V. Project HR #


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
import sys
import matplotlib

print('python:',sys.version)
print('Numpy:',np.__version__)
print('Pandas:',pd.__version__)
print('Scikitlearn:',sklearn.__version__)
print('Seaborn: ',sns.__version__)
print('matplotlib:',matplotlib.__version__)


# I. EDA #

In [ ]:
HR=pd.read_csv(r'HR-Employee-Attrition.csv')
HR


In [ ]:
#For Google Collab ONLY :
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML/HR-Employee-Attrition.csv'
HR=pd.read_csv(file_path )
HR


In [ ]:
#HR.shape
HR.info()


In [ ]:
display(HR.iloc[0])


In [ ]:
HR.isnull().any()


In [ ]:
pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)
HR.describe() #automatically ignore the categorical data


observations deduced from describe():
<ul>
<li><b>EmplyeeCount:</b>All values are 1, little information, can drop column. </li>
<li><b>EmplyeeNumber:</b>Sequential count.little information, can drop column.</li>
<li><b>StandardHours:</b>All values are 80, little information, can drop column.</li>


In [ ]:
num_col=list(HR.describe().columns)
categorical_col=list(set(HR.columns).difference(num_col))
remove_list=['EmplyeeCount','EmplyeeNumber','StandardHours']
col_numerical=[e for e in num_col if e not in remove_list]
categorical_col, col_numerical


### 2nd method ###

In [ ]:
categorical_col=[]
for k,v in HR.iteritems():
    if v.dtype == 'object':
        categorical_col.append(k)
categorical_col


In [ ]:
HR.info()
print(len(num_col))
print(len(col_numerical))
print(len(categorical_col))



## Plotting ##

In [ ]:
plt.figure(figsize=(16, 8))
h2=HR.loc[:, ['Age', 'Education', 'JobLevel','YearsAtCompany']]#,'YearsInCurrentRole']]
for col in h2.columns:
    sns.pairplot(h2, hue=col ,palette='bright', kind='scatter', markers=None,dropna=False, height=6)
plt.show()


In [ ]:
HR[col_numerical].corr()
plt.figure(figsize=(16, 8))
sns.heatmap(HR[col_numerical].corr(), annot=True, fmt='.2f')


In [ ]:
plt.figure(figsize=(20,10))
#mask for upper triangle
mask=np.zeros_like(HR[col_numerical].corr(), dtype=bool)
mask[np.triu_indices_from(mask)]=True
#Generate custom diverging colormap
cmap=sns.diverging_palette(220,10,as_cmap=True)
#Heatmap with mask with correct aspect ratio
sns.heatmap(HR[col_numerical].corr(), mask=mask, cmap=cmap, vmax=0.3,
            center=0, square=True, linewidths=0.5, cbar_kws={'shrink':0.5})
plt.show()


In [ ]:
plt.figure(figsize=(20,10))
#mask for upper triangle
mask=np.zeros_like(HR[col_numerical].corr(), dtype=bool)
mask[np.triu_indices_from(mask)]=True
#Generate custom diverging colormap
cmap=sns.light_palette((210,90,60), input='husl')
#Heatmap with mask with correct aspect ratio
sns.heatmap(HR[col_numerical].corr(), mask=mask, cmap=cmap, annot=True, fmt='.2f')
plt.show()


Correlation > 0.7 is closely correlated. We'll use 0.7 as guide into investigation:
<ul>
<li>Monthly income and Job level</li>
<li>Total working years and Job level  </li>
<li>Total working years and Monthly income   </li>
<li>Performance rating and Percent salary hike </li>    
<li>Years in current role and Years at company </li>   
<li>Years with current manager and Years at company </li>
<li>Years with current manager and Years in current role </li>
</ul>


In [ ]:
HR['Attrition'].unique()
Attrition_num={'Yes':1, 'No':0}
categorical_col

In [ ]:
HR['Attrition_num']=HR['Attrition'].map(Attrition_num)
HR['Attrition_num']


Use get_dummies or labelEncoder. <br>
Point to note:
<ul>
<li>Perform one hot encoding before train-test-split. This is fine as you are just transforming the data.There is no leakage here.</li>
<li>Perform data processing after train-test split because standardisation learn from the data by calculating mean, std etc...</li>
</ul>


In [ ]:
categorical_col.remove('Attrition')
DF_categ=pd.get_dummies(HR[categorical_col])
DF_categ


In [ ]:
X=pd.concat([HR[col_numerical], DF_categ], axis=1)
Y=HR['Attrition_num']
X


In [ ]:
Y.value_counts()
X.info()


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score

def print_score(clf, X_train, X_test, Y_train, Y_test, train=True):
    "Print the accuracy score, Classification report and confusion matrix"
    LB=preprocessing.LabelBinarizer()
    LB.fit(Y_train)
    if train:
        "Train performance"
        res=clf.predict(X_train)

        print(f'Train Results:\n')
        print(f'Accuracy Score: %.4f \n' % (accuracy_score(Y_train,res)))
        print(f'Classification report:\n {classification_report(Y_train,res)} ')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_train,res)}')
        print(f'ROC AUC Score:%.4f'%(roc_auc_score(LB.transform(Y_train),LB.transform(res))))

        res=cross_val_score(clf,X_train, Y_train, cv=100, scoring='accuracy')
        print('Average Accuracy \t: %.4f'%(np.mean(res)))
        print('Accuracy SD \t: %.4f '% (np.std(res)))

    elif train==False:
        "Test performance"
        res_test=clf.predict(X_test)
        print(f'Test Results:\n')
        print('Accuracy Score: %.4f \n'%(accuracy_score(Y_test,res_test)))
        print(f'Classification report:\n {classification_report(Y_test,res_test)}')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_test,res_test)}')
        print('ROC AUC Score: %.4f\n'%(roc_auc_score(LB.transform(Y_test),LB.transform(res_test))))
        print('Average Accuracy \t: %.4f'%(np.mean(res_test)))
        print('Accuracy SD \t: %.4f '% (np.std(res_test)))


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test, Y_train,Y_test=train_test_split(X, Y, test_size=0.3 )
X_train.shape, Y_train.shape


# II. Classification: #

## 1) Decision Tree Classifier ##

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

T=tree.DecisionTreeClassifier(random_state=42)
clf_DT=T.fit(X_train, Y_train)
clf_DT


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

score1=accuracy_score(Y_train, clf_DT.predict(X_train))
score2=classification_report(Y_train, clf_DT.predict(X_train))
score3=confusion_matrix(Y_train, clf_DT.predict(X_train))
print(f'Accuracy= {score1:.3f},\n Classification :\n{score2}, \n Confusion Matrix:\n{score3}\n')
#:.3f


In [ ]:
print_score(clf_DT, X_train, X_test, Y_train, Y_test, train=True)
print('/n--------------------------------------------------->/n')
print_score(clf_DT, X_train, X_test, Y_train, Y_test, train=False)


### Interpretation : ###
<br>
- <b>Accuracy Score: accuracy of model </b> 77%, pretty weak model <br>
- <b>Precision: error in predictions </b>  <br>
- <b>Recall: true positive rate, fraction of True cases </b>  <br>
-  <b>f1 Score: model's accuracy, fraction of True predictions </b>  <br>
-  <b>Confusion Matrix: a table of all actual values versus the predicted ones of a classifier</b>   <br>
-  <b>ROC AUC Score: measure of separability, the performance in distingishing the + and - classes </b> <br>
-  <b>Cross Validation Accuracy: k fold cross validation of the entire data set  </b>  <br>


## 2) Random Forest ##

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF_clf=RandomForestClassifier(random_state=544, n_estimators=100)
RF_clf.fit(X_train, Y_train.ravel())


In [ ]:

print(f' {print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


### Interpretation : ###
<br>
- <b>Accuracy Score: accuracy of model </b> 85% for test set, good performance <br>
- <b>Precision: error in predictions </b>  <br>
- <b>Recall: true positive rate, fraction of True cases </b> 11% for positive True Cases, which is very weak <br>
-  <b>f1 Score: model's accuracy, fraction of True predictions </b>  <br>
-  <b>Confusion Matrix: a table of all actual values versus the predicted ones of a classifier</b>   <br>
-  <b>ROC AUC Score: measure of separability, the performance in distingishing the + and - classes </b> <br>
-  <b>Cross Validation Accuracy: k fold cross validation of the entire data set  </b>  <br>


## 3) SVM ##

In [ ]:
from sklearn import svm

C=10
SVM_clf=svm.SVC(kernel='linear', C=C)
SVM_clf.fit(X_train, Y_train)


In [ ]:

print(f' {print_score(SVM_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(SVM_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


### Interpretation : ###

| Parameter	| Role	| Value|  Interpretation |
|:- |:- |:- |:- |
|<b>Accuracy Score: </b>  | 	accuracy of model   |    |    |  
|<b>Precision: </b>| error in predictions  |   |  |  
|<b>Recall:  </b>| True positive rate, fraction of True cases </b>  |  |   |  
|<b>f1 Score:  </b>|model's accuracy, fraction of True predictions  |  	 |   |
|<b>Confusion Matrix:  </b>| a table of all actual values versus the predicted ones of a classifier |   |  |
|<b>ROC AUC Score:   </b>| measure of separability, the performance in distingishing the + and - classes	 |     |   |
|<b>Cross Validation Accuracy: </b> | k fold cross validation of the entire data set	 |     |   |


## 4) Bagging ##

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag_clf=BaggingClassifier(estimator=RF_clf, n_estimators=500, bootstrap=True,
                          oob_score=False, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, Y_train.ravel())


In [ ]:
print_score(xgb_clf, X_train, X_test, Y_train, Y_test, train=True)
print('\n--------------------------------------------------->\n')
print_score(xgb_clf, X_train, X_test, Y_train, Y_test, train=False)


## 5) AdaBoost with RF ##

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
ada_clf.fit(X_train, Y_train.ravel())


In [ ]:
print_score(ada_clf, X_train, X_test, Y_train, Y_test, train=True)
print('\n--------------------------------------------------->\n')
print_score(ada_clf, X_train, X_test, Y_train, Y_test, train=False)


### Interpretation : ###

| Parameter	| Role	| Value|  Interpretation |
|:- |:- |:- |:- |
|<b>Accuracy Score: </b>  | 	accuracy of model   | 85%   |    |  
|<b>Precision: </b>| error in predictions  |   |  |  
|<b>Recall:  </b>| True positive rate, fraction of True cases </b>  |  |   |  
|<b>f1 Score:  </b>|model's accuracy, fraction of True predictions  |  19%	 |  very low positive predictions |
|<b>Confusion Matrix:  </b>| a table of all actual values versus the predicted ones of a classifier |   |  |
|<b>ROC AUC Score:   </b>| measure of separability, the performance in distingishing the + and - classes	 |     |   |
|<b>Cross Validation Accuracy: </b> | k fold cross validation of the entire data set	 |     |   |


## 6) XG boost ##

In [ ]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(max_depth=5, n_estimators=500, learning_rate=0.3, n_jobs=-1)
xgb_clf.fit(X_train, Y_train.ravel())


In [ ]:
print_score(xgb_clf, X_train, X_test, Y_train, Y_test, train=True)
print('\n--------------------------------------------------->\n')
print_score(xgb_clf, X_train, X_test, Y_train, Y_test, train=False)


### Interpretation : ###

| Parameter	| Role	| Value|  Interpretation |
|:- |:- |:- |:- |
|<b>Accuracy Score: </b>  | 	accuracy of model   | 86%   |    |  
|<b>Precision: </b>| error in predictions  |   |  |  
|<b>Recall:  </b>| True positive rate, fraction of True cases </b>  |  |   |  
|<b>f1 Score:  </b>|model's accuracy, fraction of True predictions  |  43% for +	 | quite low but the best so far  |
|<b>Confusion Matrix:  </b>| a table of all actual values versus the predicted ones of a classifier |   |  |
|<b>ROC AUC Score:   </b>| measure of separability, the performance in distingishing the + and - classes	 |     |   |
|<b>Cross Validation Accuracy: </b> | k fold cross validation of the entire data set	 |     |   |


# II. Clustering: ##

## KNN ##

## DistanceMetric class documentation
[scikit-learn]()

Metrics intended for real-valued vector spaces:

| identifier	| class name	| args	| distance function |
|:- |:- |:- |:- |
|“euclidean” | 	EuclideanDistance  |   | $\sqrt{\sum(x - y)^2)}$  |  
|“manhattan” | ManhattanDistance |  | $\sum\big|x - y\big|$|  
|“chebyshev” | ChebyshevDistance |  | max${\big|x - y\big|}$ |  
|“minkowski” | MinkowskiDistance	 | p	 | $\sum(\big|x - y\big|^p)^{\frac{1}{p}}$     |
|“wminkowski” | WMinkowskiDistance	 | p, w	 | $\sum(w\big|x - y\big|^p)^{\frac{1}{p}}$     |
|“seuclidean” | SEuclideanDistance	 | V	 | $\sqrt{\sum\frac{(x - y)^2}{V})}$     |

Refer to documentation for more on
* Metrics intended for two-dimensional vector spaces
* Metrics intended for integer-valued vector spaces
* Metrics intended for boolean-valued vector spaces

* User-defined distance

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=6,p=2, metric='euclidean')
knn.fit(X_train, Y_train)


In [ ]:
print_score(knn, X_train, X_test, Y_train, Y_test, train=True)
print('\n--------------------------------------------------->\n')
print_score(knn, X_train, X_test, Y_train, Y_test, train=False)


### Interpretation : ###

| Parameter	| Role	| Value|  Interpretation |
|:- |:- |:- |:- |
|<b>Accuracy Score: </b>  | 	accuracy of model   |    |    |  
|<b>Precision: </b>| error in predictions  |   |  |  
|<b>Recall:  </b>| True positive rate, fraction of True cases </b>  |  |   |  
|<b>f1 Score:  </b>|model's accuracy, fraction of True predictions  |  10%	 | low True positive   |
|<b>Confusion Matrix:  </b>| a table of all actual values versus the predicted ones of a classifier |   |  |
|<b>ROC AUC Score:   </b>| measure of separability, the performance in distingishing the + and - classes	 |     |   |
|<b>Cross Validation Accuracy: </b> | k fold cross validation of the entire data set	 |     |   |


# III. Dimensionality Reduction #

## PCA ##

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=3)
pca.fit(X_train)


In [ ]:
pca.components_, pca.explained_variance_, pca.explained_variance_ratio_


In [ ]:
X_pca=pca.fit_transform(X_train)
c=Y_train

plt.figure(figsize=(8,15))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=c, cmap='viridis')  # 'y' is the target variable for coloring points

plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA Scatter Plot')
plt.legend(loc='best')
plt.colorbar()  # Add a colorbar if you're using class labels for coloring
plt.show()


### Conclusion: ###
- In conclusion, seems like our models were able to predict the negative Attrition  with precision, although, all models fail to predict the positive Attrition (the best model which is built with XG boost gave f1 score for + prediction of 43% )
- The PCA helped us see a distribution of positive Attrition influenced by Incomes.